In [ ]:
class Course:
    def __init__(self, code, name, course_type, credit_hours, semester, prerequisites=None):
        self.code = code
        self.name = name
        self.course_type = course_type
        self.credit_hours = credit_hours
        self.semester = semester
        self.prerequisites = prerequisites if prerequisites is not None else []

    def add_prerequisite(self, prerequisite):
        self.prerequisites.append(prerequisite)


class KnowledgeBase:
    def __init__(self):
        self.courses = {}

    def add_course(self, course):
        self.courses[course.code] = course

    def add_prerequisite(self, course_code, prerequisite_code):
        if course_code in self.courses and prerequisite_code in self.courses:
            self.courses[course_code].add_prerequisite(prerequisite_code)
        else:
            print("Error: Course or prerequisite not found.")

    def delete_course(self, course_code):
        if course_code in self.courses:
            del self.courses[course_code]
            print(f"Course '{course_code}' deleted.")
        else:
            print("Course not found.")

    def display_courses(self):
        print("Courses in Knowledge Base:")
        for code, course in self.courses.items():
            print(f"{course.code}: {course.name} - Type: {course.course_type} - Credit Hours: {course.credit_hours} - Semester: {course.semester}")
            if course.prerequisites:
                print("Prerequisites:", ", ".join(course.prerequisites))


class Student:
    def __init__(self):
        self.semester = 0
        self.cgpa = 0.0
        self.passed_courses = []
        self.failed_courses = []

    def set_semester(self, semester):
        self.semester = semester

    def set_cgpa(self, cgpa):
        self.cgpa = cgpa

    def add_passed_course(self, course_code):
        self.passed_courses.append(course_code)

    def add_failed_course(self, course_code):
        self.failed_courses.append(course_code)


def advise_courses(student, knowledge_base):
    eligible_courses = []
    ineligible_courses = []

    
    def is_eligible(course):
        prerequisites_satisfied = all(prerequisite in student.passed_courses for prerequisite in course.prerequisites)
        return prerequisites_satisfied

    # Function to check if a course is ineligible due to failed prerequisites
    def has_failed_prerequisites(course):
        failed_prerequisites = [prerequisite for prerequisite in course.prerequisites if prerequisite not in student.passed_courses]
        return failed_prerequisites

    # Determine eligible and ineligible courses
    for code, course in knowledge_base.courses.items():
        if course.course_type != "Elective" and course.code not in student.passed_courses and course.semester <= student.semester:
            if is_eligible(course):
                eligible_courses.append(course)
            else:
                failed_prereqs = has_failed_prerequisites(course)
                if failed_prereqs:
                    ineligible_courses.append((course, failed_prereqs))

    print("\nAdvised Courses for Registration:")
    for course in eligible_courses:
        print(f"{course.code}: {course.name} - Credit Hours: {course.credit_hours}")

    print("\nIneligible Courses for Current Semester:")
    for course, failed_prereqs in ineligible_courses:
        for failed_prereq in failed_prereqs:
            print(f"{course.code}: {course.name} - Ineligible because you didn't pass or take {failed_prereq}")

    elective_courses = []
    if student.semester in [5, 6, 7, 8]:
        elective_courses = [course for course in knowledge_base.courses.values() if course.code.startswith("CSE") and course.semester == student.semester]

    print(f"\nAdvised Elective Courses for Semester {student.semester}:")
    for course in elective_courses:
        print(f"{course.code}: {course.name} - Credit Hours: {course.credit_hours}")

    return eligible_courses, elective_courses




def student_workflow(knowledge_base):
    
    student = Student()
    student.set_semester(int(input("Enter your semester: ")))
    student.set_cgpa(float(input("Enter your CGPA: ")))

    passed_courses_input = input("Enter the codes of your passed courses (separated by comma), if any: ")
    if passed_courses_input:
        passed_courses = passed_courses_input.split(",")
        for course_code in passed_courses:
            student.add_passed_course(course_code.strip())

    failed_courses_input = input("Enter the codes of your failed courses (separated by comma), if any: ")
    if failed_courses_input:
        failed_courses = failed_courses_input.split(",")
        for course_code in failed_courses:
            student.add_failed_course(course_code.strip())

    
    eligible_courses, elective_choices = advise_courses(student, knowledge_base)

    
    total_credit_hours = 0
    for course in eligible_courses:
        total_credit_hours += course.credit_hours

    if student.cgpa >= 2.0:
        if total_credit_hours > 20:
            print("\nYou have been nominated for courses exceeding the maximum credit hours (20).")
        else:
            print("\nYou have been nominated for the following courses:")
            for course in eligible_courses:
                print(f"{course.code}: {course.name} - Credit Hours: {course.credit_hours}")
    else:
        if total_credit_hours > 13:
            print("\nYou have been nominated for courses exceeding the maximum credit hours (13).")


def admin_workflow(knowledge_base):
    while True:
        print("\nAdmin Menu:")
        print("1. Add course for a specific semester")
        print("2. Delete course for a specific semester")
        print("3. Add prerequisites")
        print("4. Display courses")
        print("5. Exit")
        choice = input("Enter your choice: ")

        if choice == '1':
            add_course_for_semester(knowledge_base)
        elif choice == '2':
            delete_course_for_semester(knowledge_base)
        elif choice == '3':
            add_prerequisites(knowledge_base)
        elif choice == '4':
            knowledge_base.display_courses()
        elif choice == '5':
            break
        else:
            print("Invalid choice. Please enter a valid option.")


def add_course_for_semester(knowledge_base):
    semester = int(input("Enter the semester for which you want to add a course: "))
    code = input("Enter the code of the course: ").upper()
    name = input("Enter the name of the course: ")
    course_type = input("Enter the type of the course: ")
    credit_hours = int(input("Enter the credit hours of the course: "))
    prerequisites = input("Enter prerequisites (comma-separated if any): ").upper().split(",")
    course = Course(code, name, course_type, credit_hours, semester, prerequisites)
    knowledge_base.add_course(course)
    print(f"Course '{course.code}' added for semester {semester}.")


def delete_course_for_semester(knowledge_base):
    semester = int(input("Enter the semester from which you want to delete a course: "))
    code = input("Enter the code of the course you want to delete: ").upper()
    knowledge_base.delete_course(code)


def add_prerequisites(knowledge_base):
    while True:
        course_code = input("Enter course code to add prerequisite (or enter 'done' to finish): ").upper()
        if course_code == 'DONE':
            break
        prerequisite_code = input("Enter prerequisite code for this course: ").upper()
        knowledge_base.add_prerequisite(course_code, prerequisite_code)

def main():
    
    knowledge_base = KnowledgeBase()

    
    courses_info = [
        # University Requirements
        ("PSC101", "Human Rights and Fighting Corruption", "University Requirement", 2, 1,[]),
        ("LAN022", "English Language 1", "University Requirement", 2, 1),
        # Compulsory Courses
        ("LAN114", "Literary/Artistic Appreciation", "Compulsory", 2, 1,[]),
        ("MGT222", "Entrepreneurship and Innovation", "Compulsory", 2, 1,[]),
        ("LAN112", "Critical Thinking", "Compulsory", 2, 1,[]),
        ("GEO217", "Climate Change and Sustainability", "Compulsory", 2, 1,[]),
        ("CSE013", "Introduction to IT", "Compulsory", 2, 1,[]),
        ("LIB116", "Research and Analysis Skills", "Compulsory", 2, 1,[]),
        # Core Courses Semester 1
        ("MAT111", "Mathematics I", "Core", 3, 1,[]),
        ("PHY211", "Physics II", "Core", 3, 1,[]),
        ("CSE014", "Structured Programming", "Core", 3, 1,[]),
        ("UC1", "University Requirement (1)", "Core", 2, 1,[]),
        ("UC2", "University Requirement (2)", "Core", 2, 1,[]),
        ("UE1", "Elective University (1)", "Core", 2, 1,[]),
        # Core Courses Semester 2
        ("MAT112", "Mathematics II", "Core", 3, 2,["MAT111"]),
        ("MAT131", "Probability & Statistics I", "Core", 3, 2,[]),
        ("CSE015", "Object Oriented Programming", "Core", 3, 2,["CSE014"]),
        ("CSE315", "Discrete Mathematics 2", "Core", 3, 2,[]),
        ("UC3", "University Requirement (3)", "Core", 2, 2,[]),
        ("UE2", "Elective University (2)", "Core", 2, 2,[]),
        # Core Courses Semester 3
        ("MAT212", "Linear Algebra", "Core", 3, 3,[]),
        ("MAT231", "Probability & Statistics II", "Core", 3, 3,["MAT131"]),
        ("CSE111", "Data Structures", "Core", 3, 3,["CSE015"]),
        ("CSE131", "Logic Design", "Core", 3, 3,[]),
        ("AIE111", "Artificial Intelligence", "Core", 3, 3,["CSE015"]),
        ("UC4", "University Requirement (4)", "Core", 2, 3,[]),
        # Core Courses Semester 4
        ("MAT312", "Differential Equations", "Core", 3, 4,["MAT112"]),
        ("CSE132", "Computer Architecture & Organization", "Core", 3, 4,["CSE131"]),
        ("CSE221", "Database Systems", "Core", 3, 4,["CSE111"]),
        ("CSE211", "Web Programming", "Core", 3, 4,[]),
        ("CSE251", "Software Engineering", "Core", 3, 4,["CSE015"]),
        ("AIE191", "Field Training 1 in Computer Science", "Core", 2, 4,[]),
        # Core Courses Semester 5
        ("CSE233", "Operating Systems", "Core", 3, 5,[]),
        ("E1", "Elective Course (1)", "Core", 3, 5,[]),
        ("CSE261", "Computer Networks", "Core", 3, 5,["CSE014"]),
        ("AIE121", "Machine Learning", "Core", 3, 5,["AIE111"]),
        ("CSE352", "Systems Analysis & Design", "Core", 3, 5,["CSE111"]),
        ("UC5", "University Requirement (5)", "Core", 2, 5,[]),
        # Core Courses Semester 6
        ("CSE112", "Design & Analysis of Algorithms", "Core", 4, 6,["CSE111"]),
        ("E2", "Elective Course (2)", "Core", 3, 6,[]),
        ("CSE363", "Cloud Computing", "Core", 3, 6,[]),
        ("CSE373", "Parallel Programming", "Core", 3, 6,[]),
        ("AIE292", "Field Training 2 in Computer Science", "Core", 2, 6,["CSE191"]),
        ("UC6", "University Requirement (6)", "Core", 2, 6,[]),
        # Core Courses Semester 7
        ("CSE271", "Introduction to Parallel Computing", "Core", 3, 7,[]),
        ("CSE322", "Big Data Analytics 1", "Core", 3, 7,["CSE363"]),
        ("AIE323", "Data Mining", "Core", 3, 7,[]),
        ("AIE493", "Graduation Project 1", "Core", 3, 7,[]),
        ("E3", "Elective Course (3)", "Core", 3, 7,[]),
        ("UC7", "University Requirement (7)", "Core", 2, 7,[]),
        # Core Courses Semester 8
        ("CSE411", "Theory of Computation & Compilers", "Core", 3, 8,[]),
        ("CSE427", "Selected Topics in Big Data", "Core", 3, 8,["CSE322"]),
        ("AIE494", "Graduation Project 2", "Core", 3, 8,["CSE493"]),
        ("E4", "Elective Course (4)", "Core", 3, 8,[]),
        ("E5", "Elective Course (5)", "Core", 3, 8,[]),
        ("UE3", "Elective University (3)", "Core", 3, 8,[]),
        # Elective Courses E1
        ("CSE234", "Embedded Systems", "Elective (E1)", 3, 5,[]),
        ("CSE281", "Image Processing", "Elective (E1)", 3, 5,[]),
        ("CSE312", "Advanced Web Programming", "Elective (E1)", 3, 5,[]),
        ("CSE313", "Mobile Programming", "Elective (E1)", 3, 5,[]),
        ("CSE323", "Advanced Database Systems", "Elective (E1)", 3, 5,[]),
        # Elective Courses E2
        ("CSE446", "Information & Computer Networks Security", "Elective (E2)", 3, 6),
        ("CSE454", "Advanced Software Engineering", "Elective (E2)", 3, 6,[]),
        ("CSE464", "Internet of Things", "Elective (E2)", 3, 6,[]),
        ("AIE231", "Neural Networks", "Elective (E2)", 3, 6,[]),
        # Elective Courses E3
        ("CSE383", "Computer Vision", "Elective (E3)", 3, 7,[]),
        ("AIE241", "Natural Language Processing", "Elective (E3)", 3, 7,[]),
        ("AIE322", "Advanced Machine Learning", "Elective (E3)", 3, 7,[]),
        ("BMD311", "Introduction to Bioinformatics", "Elective (E3)", 3, 7,[]),
        ("BMD312", "Clinical Informatics", "Elective (E3)", 3, 7,[]),
        # Elective Courses E4
        ("CSE424", "Data Warehousing", "Elective (E4)", 3, 8,[]),
        ("AIE343", "Machine Learning for Text Mining", "Elective (E4)", 3, 8,[]),
        ("AIE424", "Intelligent Decision Support Systems", "Elective (E4)", 3, 8,[]),
        ("AIE425", "Intelligent Recommender Systems", "Elective (E4)", 3, 8,[]),
        ("AIE314", "AI-Based Programming", "Elective (E4)", 3, 8,[]),
        # Elective Courses E5
        ("CSE425", "Big Data Analytics 2", "Elective (E5)", 3, 8,[]),
        ("CSE426", "Selected Topics in Data Science", "Elective (E5)", 3, 8,[]),
        ("CSE474", "Distributed Information Systems", "Elective (E5)", 3, 8,[]),
        ("CSE475", "High Performance Computing", "Elective (E5)", 3, 8,[]),
        ("AIE342", "Advanced Methods for Data Analysis", "Elective (E5)", 3, 8,[]),
        ("AIE332", "Deep Learning", "Elective (E5)", 3, 8,[]),
    ]


    for course_info in courses_info:
        course = Course(*course_info)
        knowledge_base.add_course(course)

    while True:
        user_type = input("Are you a student or an admin? (Type 'student' or 'admin', or 'exit' to quit): ").lower()

        if user_type == 'student':
            student_workflow(knowledge_base)
        elif user_type == 'admin':
            admin_password = input("Enter admin password: ")
            if admin_password == 'admin12345':
                admin_workflow(knowledge_base)
            else:
                print("Incorrect password. Access denied.")
        elif user_type == 'exit':
            break
        else:
            print("Invalid input. Please choose 'student' or 'admin', or 'exit' to quit.")


if __name__ == "__main__":
    main()


Are you a student or an admin? (Type 'student' or 'admin', or 'exit' to quit):  student
Enter your semester:  5
Enter your CGPA:  2.42
Enter the codes of your passed courses (separated by comma), if any:   PSC101, LAN022, LAN114, MGT222, LAN112, GEO217, CSE013, LIB116, AN122, ACC113, DVA014, MEC013, ARC010, SOC107, MAT111, PHY211, CSE014, UC1, UC2, UE1, MAT112, MAT131, CSE015, CSE315, UC3, UE2
Enter the codes of your failed courses (separated by comma), if any:  



Advised Courses for Registration:
MAT212: Linear Algebra - Credit Hours: 3
MAT231: Probability & Statistics II - Credit Hours: 3
CSE111: Data Structures - Credit Hours: 3
CSE131: Logic Design - Credit Hours: 3
AIE111: Artificial Intelligence - Credit Hours: 3
UC4: University Requirement (4) - Credit Hours: 2
MAT312: Differential Equations - Credit Hours: 3
CSE211: Web Programming - Credit Hours: 3
CSE251: Software Engineering - Credit Hours: 3
AIE191: Field Training 1 in Computer Science - Credit Hours: 2
CSE233: Operating Systems - Credit Hours: 3
E1: Elective Course (1) - Credit Hours: 3
CSE261: Computer Networks - Credit Hours: 3
UC5: University Requirement (5) - Credit Hours: 2
CSE234: Embedded Systems - Credit Hours: 3
CSE281: Image Processing - Credit Hours: 3
CSE312: Advanced Web Programming - Credit Hours: 3
CSE313: Mobile Programming - Credit Hours: 3
CSE323: Advanced Database Systems - Credit Hours: 3

Ineligible Courses for Current Semester:
CSE132: Computer Architecture & 